In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import v2
import numpy as np

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 16
batch_size = 8
learning_rate = 0.001

transform = v2.Compose(
    [v2.ToTensor(),
     v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

tarin_loder = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


/Users/orifmilod/Desktop/projects/gigatorch/env/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

model = ConvNet()
model.to(device)
critertion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(tarin_loder)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(tarin_loder):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = critertion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoche [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print("Finished training")


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')


Epoche [1/16], Step [2000/6250], Loss: 2.3213
Epoche [1/16], Step [4000/6250], Loss: 2.2904
Epoche [1/16], Step [6000/6250], Loss: 2.2973
Epoche [2/16], Step [2000/6250], Loss: 2.2808
Epoche [2/16], Step [4000/6250], Loss: 2.1523
Epoche [2/16], Step [6000/6250], Loss: 2.2306
Epoche [3/16], Step [2000/6250], Loss: 2.3663
Epoche [3/16], Step [4000/6250], Loss: 2.1092
Epoche [3/16], Step [6000/6250], Loss: 1.2913
Epoche [4/16], Step [2000/6250], Loss: 1.6886
Epoche [4/16], Step [4000/6250], Loss: 2.6763
Epoche [4/16], Step [6000/6250], Loss: 1.7113
Epoche [5/16], Step [2000/6250], Loss: 1.2147
Epoche [5/16], Step [4000/6250], Loss: 1.8217
Epoche [5/16], Step [6000/6250], Loss: 1.5523
Epoche [6/16], Step [2000/6250], Loss: 1.2940
Epoche [6/16], Step [4000/6250], Loss: 1.6208
Epoche [6/16], Step [6000/6250], Loss: 1.4940
Epoche [7/16], Step [2000/6250], Loss: 1.7596
Epoche [7/16], Step [4000/6250], Loss: 1.1859
Epoche [7/16], Step [6000/6250], Loss: 1.7896
Epoche [8/16], Step [2000/6250], L

IndexError: index 8 is out of bounds for dimension 0 with size 8